In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -r requirements19.txt

# **Set the file paths and notes**

<p>Use option = 0 if you want to do training and validation using 2 of the folds (keeping the 3rd fold for final training and testing).</p>

<p>Use option = 1 if you want to do final training (using 2 folds) and testing (with the 3rd fold).</p>

Be sure to update the name of the files - doing 3 cross-validation in this project is done manually with the prepared data to create comparable results with the rest of the projects. See an example below.

Due to the nature of this team's model, early stopping is carried out by manual experimentation during testing and validation for 3CV. To help reach a decision on the best number of epochs for training - refer to the tables with the loss of the generator and discriminator.


In [ ]:
option = 1

# path for training and validation (without using the 3rd fold). This is an example from the ClusterCV folder
path_lr_data_tr = 'lr_split_AandB_training.csv'
path_hr_data_tr = 'hr_split_AandB_training.csv'
path_lr_data_valid = 'lr_split_AandB_validation.csv'
path_hr_data_valid = 'hr_split_AandB_validation.csv'

# path for final training and testing (using fold A and B for training and C for testing).This is an example from the ClusterCV folder
path_lr_data = 'lr_split_AandB_finaltraining.csv'
path_hr_data = 'hr_split_AandB_finaltraining.csv'
path_lr_data_test = 'lr_clusterC.csv'
path_hr_data_test = 'hr_clusterC.csv'

# path to save the evaluation metrics to - this means that Cluster C will be used for testing
path_eval_matrics = '19-clusterCV-split3.csv'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GINConv

"""
Define layers:
    GraphConvolution
    GIN
    GAT
    GCN
    GraphUnpool
    GraphPool
    Dense
"""

class GraphConvolution(nn.Module):
    """
    Simple Graph Convolutional Network (GCN) layer.
    """

    def __init__(self, in_features, out_features, dropout, act=F.relu):
        """
        Initialize the GraphConvolution layer.

        Args:
            in_features (int): Number of input features.
            out_features (int): Number of output features.
            dropout (float): Dropout probability.
            act (function, optional): Activation function, default is ReLU.
        """
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.weight = torch.nn.Parameter(
            torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        """
        Resets the parameters using Xavier uniform initialization.
        """
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        """
        Forward pass of the GraphConvolution layer.

        Args:
            input (torch.Tensor): Input matrix.
            adj (torch.Tensor): Adjacency matrix.

        Returns:
            output (torch.Tensor): Aggregation output.
        """
        input = F.dropout(input, self.dropout, self.training)
        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        output = self.act(output)
        return output

class GIN(nn.Module):
    """
    Graph Isomorphism Network (GIN) module.
    """

    def __init__(self, in_dim, out_dim):
        """
        Initialize the GIN module.

        Args:
            in_dim (int): Dimensionality of input features.
            out_dim (int): Dimensionality of output features.
        """
        super(GIN, self).__init__()
        self.gin_conv = GINConv(nn.Linear(in_dim, out_dim))
        self.act = nn.ReLU()

    def forward(self, A, X):
        """
        Forward pass of the GIN module.

        Args:
            A (torch.Tensor): The adjacency matrix.
            X (torch.Tensor): The input matrix.

        Returns:
            X (torch.Tensor): The output matrix after GIN convolution.
        """
        edge_index = convert_adj_to_edge_index(A)
        X = self.gin_conv(X, edge_index)
        X = self.act(X)

        return X

class GAT(nn.Module):
    """
    Graph Attention Network (GAT) module.
    """

    def __init__(self, in_dim, out_dim):
        """
        Initialize the GIN module.

        Args:
            in_dim (int): Dimensionality of input features.
            out_dim (int): Dimensionality of output features.
        """
        super(GAT, self).__init__()
        self.gat_conv = GATConv(in_dim, out_dim)
        self.act = nn.ReLU()

    def forward(self, A, X):
        """
        Forward pass of the GIN module.

        Args:
            A (torch.Tensor): The adjacency matrix.
            X (torch.Tensor): The input matrix.

        Returns:
            X (torch.Tensor): The output matrix after GIN convolution.
        """
        edge_index = convert_adj_to_edge_index(A)
        X = self.gat_conv(X, edge_index)
        X = self.act(X)
        # Add dropout
        X = F.dropout(X, p=0.6, training=self.training)

        return X

class GCN(nn.Module):
    """
    Graph Convolution Network (GCN) module.
    """
    def __init__(self, in_dim, out_dim):
        """
        Initialize the GCN module.

        Args:
            in_dim (int): Dimensionality of input features.
            out_dim (int): Dimensionality of output features.
        """
        super(GCN, self).__init__()
        self.proj = nn.Linear(in_dim, out_dim)
        self.drop = nn.Dropout(p=0.6)
        self.act = nn.ReLU()

    def forward(self, A, X):
        """
        Forward pass of the GCN module.

        Args:
            A (torch.Tensor): The adjacency matrix.
            X (torch.Tensor): The input matrix.

        Returns:
            X (torch.Tensor): The output matrix after GCN convolution.
        """

        X = self.drop(X)
        X = torch.matmul(A, X)
        X = self.proj(X)
        X = self.act(X) # activation
        return X

class GraphUnpool(nn.Module):
    """
    Graph Unpooling layer module.
    """

    def __init__(self):
        """
        Initialize the GraphUnpool layer.
        """
        super(GraphUnpool, self).__init__()

    def forward(self, A, X, idx):
        """
        Forward pass of the GraphUnpool layer.

        Args:
            A (torch.Tensor): The adjacency matrix.
            X (torch.Tensor): The node embedding matrix.
            idx (torch.Tensor): Nodes to be unpolled.

        Returns:
            A (torch.Tensor): The unpolled adjacency matrix.
            new_X (torch.Tensor): The unpolled node embedding matrix.
        """
        new_X = torch.zeros([A.shape[0], X.shape[1]])
        new_X[idx] = X
        return A, new_X

class GraphPool(nn.Module):
    """
    Graph Pooling layer module.
    """

    def __init__(self, k, in_dim):
        """
        Initialize the GraphPool layer.

        Args:
            k (float): The ratio of nodes to keep after pooling.
            in_dim (int): Dimensionality of input features.
        """
        super(GraphPool, self).__init__()
        self.k = k
        self.proj = nn.Linear(in_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, A, X):
        """
        Forward pass of the GraphPool layer.

        Args:
            A (torch.Tensor): The adjacency matrix.
            X (torch.Tensor): The node embedding matrix.

        Returns:
            A (torch.Tensor): The pooled adjacency matrix.
            new_X (torch.Tensor): The pooled node embedding matrix.
            idx (torch.Tensor): Indices of the nodes selected during pooling.
        """
        scores = self.proj(X)
        scores = torch.squeeze(scores)
        scores = self.sigmoid(scores/100)
        num_nodes = A.shape[0]
        values, idx = torch.topk(scores, int(self.k*num_nodes))
        new_X = X[idx, :]
        values = torch.unsqueeze(values, -1)
        new_X = torch.mul(new_X, values)
        A = A[idx, :]
        A = A[:, idx]
        return A, new_X, idx

class Dense(nn.Module):
    """
    Dense layer module.
    """
    def __init__(self, n1, n2, args):
        """
        Initialize the Dense layer module.

        Args:
            n1 (int): Number of input features.
            n2 (int): Number of output features.
            args (object): Arguments object containing hyperparameters.
        """
        super(Dense, self).__init__()
        self.weights = torch.nn.Parameter(
            torch.FloatTensor(n1, n2), requires_grad=True)
        nn.init.normal_(self.weights, mean=args.mean_dense, std=args.std_dense)

    def forward(self, x):
        """
        Forward pass of the Dense (fully-connected) layer module.

        Args:
            x (torch.Tensor): Input feature matrix.

        Returns:
            out (torch.Tensor): Output feature matrix after dense linear transformation.
        """
        out = torch.mm(x, self.weights) # linear transformation
        return out

In [ ]:
import numpy as np

class MatrixVectorizer:
    """
    A class for transforming between matrices and vector representations.

    This class provides methods to convert a symmetric matrix into a vector (vectorize)
    and to reconstruct the matrix from its vector form (anti_vectorize), focusing on
    vertical (column-based) traversal and handling of elements.
    """

    def __init__(self):
        """
        Initializes the MatrixVectorizer instance.

        The constructor currently does not perform any actions but is included for
        potential future extensions where initialization parameters might be required.
        """
        pass

    @staticmethod
    def vectorize(matrix, include_diagonal=False):
        """
        Converts a matrix into a vector by vertically extracting elements.

        This method traverses the matrix column by column, collecting elements from the
        upper triangle, and optionally includes the diagonal elements immediately below
        the main diagonal based on the include_diagonal flag.

        Args:
            matrix (numpy.ndarray): The matrix to be vectorized.
            include_diagonal (bool, optional): Flag to include diagonal elements in the vectorization.
                                                Defaults to False.

        Returns:
            numpy.ndarray: The vectorized form of the matrix.
        """
        # Determine the size of the matrix based on its first dimension
        matrix_size = matrix.shape[0]

        # Initialize an empty list to accumulate vector elements
        vector_elements = []

        # Iterate over columns and then rows to collect the relevant elements
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Collect upper triangle elements
                        vector_elements.append(matrix[row, col])
                    elif include_diagonal and row == col + 1:
                        # Optionally include the diagonal elements immediately below the diagonal
                        vector_elements.append(matrix[row, col])

        return np.array(vector_elements)

    @staticmethod
    def anti_vectorize(vector, matrix_size, include_diagonal=False):
        """
        Reconstructs a matrix from its vector form, filling it vertically.

        The method fills the matrix by reflecting vector elements into the upper triangle
        and optionally including the diagonal elements based on the include_diagonal flag.

        Args:
            vector (numpy.ndarray): The vector to be transformed into a matrix.
            matrix_size (int): The size of the square matrix to be reconstructed.
            include_diagonal (bool, optional): Flag to include diagonal elements in the reconstruction.
                                                Defaults to False.

        Returns:
            numpy.ndarray: The reconstructed square matrix.
        """
        # Initialize a square matrix of zeros with the specified size
        matrix = np.zeros((matrix_size, matrix_size))

        # Index to keep track of the current position in the vector
        vector_idx = 0

        # Fill the matrix by iterating over columns and then rows
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Reflect vector elements into the upper triangle and its mirror in the lower triangle
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1
                    elif include_diagonal and row == col + 1:
                        # Optionally fill the diagonal elements after completing each column
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1

        return matrix

In [ ]:
import torch
import numpy as np

def weight_variable_glorot(output_dim):
    """
    Initialize weights using Xavier initialization.

    Args:
        output_dim (int): The number of output dimensions for the weight matrix.

    Returns:
        initial (numpy.ndarray): Randomly initialized square weight matrix.
    """

    input_dim = output_dim
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = np.random.uniform(-init_range, init_range,
                                (input_dim, output_dim))

    return initial

def convert_adj_to_edge_index(adj):
    """
    Convert adjacency matrix to edge index representation.

    Args:
        adj (torch.Tensor): Adjacency matrix.

    Returns:
        edge_index (torch.Tensor): Edge index representation of the graph.
    """
    # Assuming adj is a square matrix
    # Find non-zero elements in the adjacency matrix and
    # transpose the result for edge index representation
    edge_index = torch.nonzero(adj, as_tuple=False).t().contiguous()
    return edge_index

def pad_HR_adj(label, split):
    """
    Pad the HR matrix with zero padding.

    Args:
        label (numpy.ndarray): The HR matrix.
        split (int): The amount of padding to add to each edge of the matrix.

    Returns:
        label (numpy.ndarray): The padded HR adjacency matrix.
    """

    label = np.pad(label, ((split, split), (split, split)), mode="constant")
    # Set diagonal elements to 1
    np.fill_diagonal(label, 1)
    return label

def unpad(data, split):
    """
    Unpad the padded matrix.

    Args:
        data (numpy.ndarray): The padded matrix .
        split (int): The amount of padding to be removed from each edge of the array.

    Returns:
        train (numpy.ndarray): The unpadded matrix.
    """

    idx_0 = data.shape[0]-split
    idx_1 = data.shape[1]-split
    train = data[split:idx_0, split:idx_1]
    return train

def normalize_adj_torch(mx):
    """
    Normalize the input adjacency matrix using Kipf normalization.

    Args:
        mx (torch.Tensor): The input adjacency matrix.

    Returns:
        mx (torch.Tensor): The normalized adjacency matrix.
    """
    rowsum = mx.sum(1) # calculate the row sums
    r_inv_sqrt = torch.pow(rowsum, -0.5).flatten()
    r_inv_sqrt[torch.isinf(r_inv_sqrt)] = 0. # replace any infinite values with 0
    r_mat_inv_sqrt = torch.diag(r_inv_sqrt)

    # Perform Kipf normalization
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    mx = torch.transpose(mx, 0, 1)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    return mx

def gaussian_noise_layer(input_layer, args):
    """
    Apply Gaussian noise to the input layer.

    Args:
        input_layer (torch.Tensor): Input tensor to which Gaussian noise will be added.
        args (object): Arguments object containing hyperparameters.

    Returns:
        z (torch.Tensor): Output tensor with Gaussian noise.
    """
    z = torch.empty_like(input_layer)
    noise = z.normal_(mean=args.mean_gaussian, std=args.std_gaussian)
    z = torch.abs(input_layer + noise)

    z = (z + z.t())/2
    z = z.fill_diagonal_(1)
    return z

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


"""
Define core blocks:
    GraphUnet
    GSRLayer
    IWASAGSRNet
    Discriminator
"""

class GraphUnet(nn.Module):
    """
    Graph U-Net module.
    """
    def __init__(self, ks, in_dim, out_dim, dim=320):
        """
        Initialize the GraphUnet module.

        Args:
            ks (list): List of pooling ratios for each down-sampling step.
            in_dim (int): Dimensionality of input features.
            out_dim (int): Dimensionality of output features.
            dim (int): Dimensionality of hidden features (default is 320).
        """
        super(GraphUnet, self).__init__()
        self.ks = ks

        # Using GAT
        self.start_gat = GAT(in_dim, dim)
        self.bottom_gat = GAT(dim, dim)
        self.end_gat = GAT(2*dim, out_dim)
        self.down_gcns = []
        self.up_gcns = []
        self.pools = []
        self.unpools = []
        self.l_n = len(ks)
        for i in range(self.l_n):
            self.down_gcns.append(GCN(dim, dim))
            self.up_gcns.append(GCN(dim, dim))
            self.pools.append(GraphPool(ks[i], dim))
            self.unpools.append(GraphUnpool())

    def forward(self, A, X):
        """
        Forward pass of the GraphUnet module.

        Args:
            A (torch.Tensor): The adjacency matrix of the input graph.
            X (torch.Tensor): The input node embedding matrix.

        Returns:
            X (torch.Tensor): The output node embedding matrix.
            start_gat_outs (torch.Tensor): The start GAT layer output.
        """
        adj_ms = [] # list of adjacency matrices at each down-sampling step
        indices_list = [] # list of indices for unpooling
        down_outs = [] # list of node embedding matrices at each down-sampling step

        # Initial GAT convolution
        X = self.start_gat(A, X)
        start_gat_outs = X
        org_X = X # original node embedding matrix

        # Build stacked U-Net
        for _ in range(2):
            for i in range(self.l_n):
                X = self.down_gcns[i](A, X) # down-sampling with GCN
                adj_ms.append(A)
                down_outs.append(X)
                A, X, idx = self.pools[i](A, X) # pooling
                indices_list.append(idx)

            X = self.bottom_gat(A, X)

            for i in range(self.l_n):
                up_idx = self.l_n - i - 1 # index for unpooling
                A, idx = adj_ms[up_idx], indices_list[up_idx]
                A, X = self.unpools[i](A, X, idx) # unpooling
                X = self.up_gcns[i](A, X) # up-sampling with GCN
                X = X.add(down_outs[up_idx])

            X = torch.cat([X, org_X], 1)
            # Final GAT convolution
            X = self.end_gat(A, X)

        return X, start_gat_outs

class GSRLayer(nn.Module):
    """
    Graph Super Resolution module.
    """

    def __init__(self, hr_dim):
        """
        Initialize the GSRLayer.

        Args:
            hr_dim (int): Dimension of the high-resolution (HR) matrix.
        """
        super(GSRLayer, self).__init__()

        # Initialize weight matrix
        self.weights = torch.from_numpy(
            weight_variable_glorot(hr_dim)).type(torch.FloatTensor)
        self.weights = torch.nn.Parameter(
            data=self.weights, requires_grad=True)

    def forward(self, A, X):
        """
        Forward pass of the GSRLayer.

        Args:
            A (torch.Tensor): Adjacency matrix of the low-resolution (LR) graph.
            X (torch.Tensor): The input node embedding matrix.

        Returns:
            adj (torch.Tensor): The super-resolved graph structure.
            torch.abs(X) (torch.Tensor): The super-resolved graph node features.
        """
        with torch.autograd.set_detect_anomaly(True):

            # Extract LR adjacency matrix and dimension
            lr = A
            lr_dim = lr.shape[0]

            # Extract node embedding matrix
            f = X

            # Eigenvector decomposition of the LR adjacency matrix
            eig_val_lr, U_lr = torch.linalg.eigh(lr, UPLO='U')

            # Generate S_d (the transposed concatenation of 2 identity matrices)
            eye_mat = torch.eye(lr_dim).type(torch.FloatTensor)
            s_d = torch.cat((eye_mat, eye_mat), 0)

            # Super-resolving the graph structure
            a = torch.matmul(self.weights, s_d)
            b = torch.matmul(a, torch.t(U_lr))
            f_d = torch.matmul(b, f)
            f_d = torch.abs(f_d)
            f_d = f_d.fill_diagonal_(1)
            adj = f_d

            # Super-resolving the graph node features
            X = torch.mm(adj, adj.t())
            X = (X + X.t())/2
            X = X.fill_diagonal_(1)

        return adj, torch.abs(X)

class IWASAGSRNet(nn.Module):
    """
    Graph Super-Resolution Network (IWAS-AGSRNet) module.
    """

    def __init__(self, args):
        """
        Initialize the IWAS-AGSRNet module.

        Args:
            args (object): Arguments of input hyperparameters.
        """
        super(IWASAGSRNet, self).__init__()

        self.lr_dim = args.lr_dim
        self.hr_dim = args.hr_dim
        self.hidden_dim = args.hidden_dim

        # Initialize graph layers
        self.layer = GSRLayer(self.hr_dim)
        self.net = GraphUnet(args.ks, self.lr_dim, self.hr_dim)
        self.gc1 = GraphConvolution(
            self.hr_dim, self.hidden_dim, args.dropout, act=F.relu)
        self.gc2 = GraphConvolution(
            self.hidden_dim, self.hr_dim, args.dropout, act=F.relu)
        self.gin = GIN(self.hr_dim, self.hr_dim)

    def forward(self, lr, lr_dim):
        """
        Forward pass of the IWASAGSRNet module.

        Args:
            lr (torch.Tensor): LR adjacency matrix.
            lr_dim (int): Dimensionality of the LR input features.

        Returns:
            z (torch.Tensor): Generated HR adjacency matrix.
            unet_outs (torch.Tensor): The output node embedding matrix from the stacked U-Net.
            start_gat_outs (torch.Tensor): The output node embedding matrix from the start GAT layer from the stacked U-Net.
        """
        with torch.autograd.set_detect_anomaly(True):
            # Initialize the node embedding matrix to be identity matrix
            I = torch.eye(lr_dim).type(torch.FloatTensor)
            # Normalize LR adjacency matrix
            A = normalize_adj_torch(lr).type(torch.FloatTensor)

            # Pass LR graph through stacked U-Net
            unet_outs, start_gat_outs = self.net(A, I)

            # Generate HR graph using GSRLayer
            outputs, Z = self.layer(A, unet_outs)

            # Perform GCN and GIN to aggregate information
            hidden1 = self.gc1(Z, outputs)
            hidden2 = self.gc2(hidden1, outputs)
            z = self.gin(hidden2, outputs)

            # Symmetrize the output adjacency matrix
            z = (z + z.t())/2
            z = z.fill_diagonal_(1)

            # Post-process the output to be non-negative values
            z = torch.abs(z)

        return z, unet_outs, start_gat_outs

class Discriminator(nn.Module):
    """
    Discriminator module.
    """
    def __init__(self, args):
        """
        Initialize the Discriminator module.

        Args:
            args (object): Arguments object containing hyperparameters.
        """
        super(Discriminator, self).__init__()

        # Initialize dense layers and activation functions
        self.dense_1 = Dense(args.hr_dim, args.hr_dim, args)
        self.relu_1 = nn.LeakyReLU(0.1, inplace=False)
        self.dense_2 = Dense(args.hr_dim, args.hr_dim, args)
        self.relu_2 = nn.LeakyReLU(0.1, inplace=False)
        self.dense_3 = Dense(args.hr_dim, 1, args)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        """
        Forward pass of the Discriminator module.

        Args:
            inputs (torch.Tensor): Input feature matrix (generated HR matrix).

        Returns:
            output (torch.Tensor): Probability scores indicating real or fake samples.
        """
        dc_den1 = self.relu_1(self.dense_1(inputs))
        dc_den2 = self.relu_2(self.dense_2(dc_den1))
        output = self.sigmoid(self.dense_3(dc_den2))
        output = torch.abs(output)
        return output

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

"""
Define training and testing functions
"""

def train(model, subjects_adj, subjects_labels, args):
    """
    Train the IWAS-AGSRNet model.

    Args:
        model (nn.Module): IWAS-AGSRNet model to be trained.
        subjects_adj (list): Training set.
        subjects_labels (list): Training labels.
        args (object): Arguments object containing hyperparameters.
    """

    # Define criterions to calculate loss
    mse_criterion = nn.MSELoss() # for IWAS-AGSRNet
    bce_criterion = nn.BCELoss() # for Discriminator

    # Initialize Discriminator
    netD = Discriminator(args)

    # Prepare optimizers
    optimizerG = optim.Adam(model.parameters(), lr=args.lr_d, betas=(0.5, 0.999))
    optimizerD = optim.Adam(netD.parameters(), lr=args.lr_g, betas=(0.5, 0.999))

    # List for loss in all epochs
    all_epochs_loss = []
    for epoch in range(args.epochs):
        with torch.autograd.set_detect_anomaly(True):
            epoch_loss = []
            epoch_error = []
            epoch_gen = []
            epoch_mse = []
            epoch_dc = []
            epoch_unet = []

            for lr, hr in zip(subjects_adj, subjects_labels):
                optimizerD.zero_grad()
                optimizerG.zero_grad()

                # Process input LR and HR matrices
                hr = pad_HR_adj(hr, args.padding) # pad input HR adj matrix
                # Convert to tensor
                lr = torch.from_numpy(lr).type(torch.FloatTensor)
                padded_hr = torch.from_numpy(hr).type(torch.FloatTensor)

                # Eigenvector decomposition of HR matrix
                _, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

                # Pass through IWAS-AGSRNet to generate outputs
                model_outputs, net_outs, start_gat_outs = model(lr, args.lr_dim)

                # Define MSE loss
                mse_loss = args.lmbda * mse_criterion(net_outs, start_gat_outs) + mse_criterion(
                    model.layer.weights, U_hr) + mse_criterion(model_outputs, padded_hr)

                # Calculate MSE loss between the generated and real HR matrices
                error = mse_criterion(model_outputs, padded_hr)

                # Train Discriminator
                real_data = model_outputs.detach()
                fake_data = gaussian_noise_layer(padded_hr, args)

                d_real = netD(real_data)
                d_fake = netD(fake_data)

                dc_loss_real = bce_criterion(d_real, torch.ones(args.hr_dim, 1))
                dc_loss_fake = bce_criterion(d_fake, torch.zeros(args.hr_dim, 1))
                dc_loss = dc_loss_real + dc_loss_fake

                dc_loss.backward()
                optimizerD.step()

                d_fake = netD(gaussian_noise_layer(padded_hr, args))

                gen_loss = bce_criterion(d_fake, torch.ones(args.hr_dim, 1))
                generator_loss = gen_loss + mse_loss
                generator_loss.backward()
                optimizerG.step()

                epoch_loss.append(generator_loss.item())
                epoch_error.append(error.item())

                # Append other losses for monitoring
                unet_loss = args.lmbda * mse_criterion(net_outs, start_gat_outs)
                epoch_gen.append(gen_loss.item())
                epoch_mse.append(mse_loss.item())
                epoch_dc.append(dc_loss.item())
                epoch_unet.append(unet_loss.item())

            print("Epoch: ", epoch, "Loss: ", np.mean(epoch_loss),
                  "Error: ", np.mean(epoch_error)*100, "%",
                  "UNet Loss: ", np.mean(epoch_unet),
                  "DC Loss: ", np.mean(epoch_dc),
                  "Gen Loss: ", np.mean(epoch_gen),
                  "MSE Loss: ", np.mean(epoch_mse))

            all_epochs_loss.append(np.mean(epoch_loss))

def test(model, test_adj, test_labels, args):
    """
    Make prediction on a test set to generate HR matrix for every test sample.

    Args:
        model (torch.nn.Module): The neural network model to be evaluated.
        test_adj (list): List of adjacency matrices representing the test set.
        test_labels (list): List of ground truth matrices corresponding to the test set.
        args (Argument): An Argument object containing various parameters.

    Returns:
        pred_matrices (numpy.ndarray): Array containing the model predictions for each test sample.
        gt_matrices (numpy.ndarray): Array containing the ground truth matrices for each test sample.
    """
    # set model to evaluation mode
    model.eval()

    # Loss function
    criterion = nn.MSELoss()
    criterion_mae = nn.L1Loss()

    # Initializations
    pred_matrices_list = []
    gt_matrices_list = []
    test_error = []
    test_mae_padded = []
    test_mae_unpadded = []

    with torch.no_grad():
        for lr, hr in zip(test_adj, test_labels):
            # check whether lr and hr are empty matrix
            all_zeros_lr = not np.any(lr)
            all_zeros_hr = not np.any(hr)

            if all_zeros_lr == True or all_zeros_hr == True:
                print(all_zeros_lr, all_zeros_hr)

            if all_zeros_lr == False and all_zeros_hr == False:
                # Pad hr from 268 to 320
                lr = torch.from_numpy(lr).type(torch.FloatTensor)
                np.fill_diagonal(hr, 1)
                hr_padded = pad_HR_adj(hr, args.padding)
                hr_padded = torch.from_numpy(hr_padded).type(torch.FloatTensor)

                # Forward pass to make prediction
                preds, _, _ = model(lr, args.lr_dim)

                # Store errors
                error = criterion(preds, hr_padded)
                test_error.append(error.item())

                # Store unpadded predictoins and ground truth for plots
                preds_unpadded = unpad(preds, args.padding)
                hr_unpadded = torch.from_numpy(hr).type(torch.FloatTensor)
                pred_matrices_list.append(preds_unpadded.numpy())
                gt_matrices_list.append(hr_unpadded.numpy())

                # Use torch L1 norm to calculate MAE (padded & unpadded)
                test_mae_padded.append(criterion_mae(preds, hr_padded).item())
                test_mae_unpadded.append(criterion_mae(preds_unpadded, hr_unpadded).item())

    pred_matrices = np.stack(pred_matrices_list, axis=0)
    gt_matrices = np.stack(gt_matrices_list, axis=0)

    print("Test error MSE: ", np.mean(test_error))
    print("Test MAE (using L1Loss, padded): ", np.mean(test_mae_padded))
    print("Test MAE (using L1Loss, unpadded): ", np.mean(test_mae_unpadded))
    print("====================================================")

    return pred_matrices, gt_matrices

In [ ]:
import torch
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon

def calculate_measures(num_test_samples, num_roi, pred_matrices, gt_matrices):
    """
    Calculate and evaluate various evaluation meatures.

    Args:
        num_test_samples (int): Number of samples in the test set.
        num_roi (int): Number of regions of interest in the adjacency matrices.
        pred_matrices (numpy.ndarray): Array containing predicted adjacency matrices for each test sample.
        gt_matrices (numpy.ndarray): Array containing ground truth adjacency matrices for each test sample.

    Returns:
    - measures (list): List containing computed performance measures, including MAE, PCC, Jensen-Shannon Distance,
                      and average MAE for betweenness centrality, eigenvector centrality, and PageRank centrality.
    """

    # post-processing
    pred_matrices[pred_matrices < 0] = 0
    gt_matrices[gt_matrices < 0] = 0

    # Initialize lists to store MAEs for each centrality measure
    mae_bc = []
    mae_ec = []
    mae_pc = []

    pred_1d_list = []
    gt_1d_list = []

    # Iterate over each test sample
    for i in range(num_test_samples):
        # Convert adjacency matrices to NetworkX graphs
        pred_graph = nx.from_numpy_array(pred_matrices[i], edge_attr="weight")
        gt_graph = nx.from_numpy_array(gt_matrices[i], edge_attr="weight")

        # Compute centrality measures
        pred_bc = nx.betweenness_centrality(pred_graph, weight="weight")
        pred_ec = nx.eigenvector_centrality(pred_graph, weight="weight")
        pred_pc = nx.pagerank(pred_graph, weight="weight")

        gt_bc = nx.betweenness_centrality(gt_graph, weight="weight")
        gt_ec = nx.eigenvector_centrality(gt_graph, weight="weight")
        gt_pc = nx.pagerank(gt_graph, weight="weight")

        # Convert centrality dictionaries to lists
        pred_bc_values = list(pred_bc.values())
        pred_ec_values = list(pred_ec.values())
        pred_pc_values = list(pred_pc.values())

        gt_bc_values = list(gt_bc.values())
        gt_ec_values = list(gt_ec.values())
        gt_pc_values = list(gt_pc.values())

        # Compute MAEs
        mae_bc.append(mean_absolute_error(pred_bc_values, gt_bc_values))
        mae_ec.append(mean_absolute_error(pred_ec_values, gt_ec_values))
        mae_pc.append(mean_absolute_error(pred_pc_values, gt_pc_values))

        # Vectorize matrices
        pred_1d_list.append(MatrixVectorizer.vectorize(pred_matrices[i]))
        gt_1d_list.append(MatrixVectorizer.vectorize(gt_matrices[i]))

    # Compute average MAEs
    avg_mae_bc = sum(mae_bc) / len(mae_bc)
    avg_mae_ec = sum(mae_ec) / len(mae_ec)
    avg_mae_pc = sum(mae_pc) / len(mae_pc)

    # Concatenate flattened matrices
    pred_1d = np.concatenate(pred_1d_list)
    gt_1d = np.concatenate(gt_1d_list)

    # Compute metrics
    mae = mean_absolute_error(pred_1d, gt_1d)
    pcc = pearsonr(pred_1d, gt_1d)[0]
    js_dis = jensenshannon(pred_1d, gt_1d)

    print("MAE: ", mae)
    print("PCC: ", pcc)
    print("Jensen-Shannon Distance: ", js_dis)
    print("Average MAE PageRank centrality:", avg_mae_pc)
    print("Average MAE eigenvector centrality:", avg_mae_ec)
    print("Average MAE betweenness centrality:", avg_mae_bc)


    return [mae, pcc, js_dis, avg_mae_pc, avg_mae_ec, avg_mae_bc]


def plot_measures(measures = None, fold_index = None, avg = False, mean_values = None, std_dev_values = None):
    """
    Plot and visualize evaluation measures(MAE', 'PCC', 'JSD', 'MAE(EC)', 'MAE(BC)') across folds or for a specific fold.

    Args:
        measures (list): List of numerical values representing evaluation measures for each category.
        fold_index (int or None): Index of the fold if plotting for a specific fold, None for plotting across all folds.
        avg (bool): Flag indicating whether to plot averages across folds or for a specific fold.
        mean_values (list or None): List of mean values for each category across folds.
        std_dev_values (list or None): List of standard deviation values for each category across folds.

    Returns:
        None: The function generates and saves the plot based on the specified parameters.
    """
    # Initialization
    categories = ['MAE', 'PCC', 'JSD', 'MAE(EC)', 'MAE(BC)']
    colors = ['blue', 'green', 'red', 'purple', 'pink']

    if avg:
        # Plot across all folds
        plt.figure(4)
        mean_values = np.delete(mean_values, 3)
        std_dev_values = np.delete(std_dev_values, 3)
        plt.bar(range(len(mean_values)), mean_values, yerr=std_dev_values, capsize=5, color=colors)
        plt.xticks(range(len(mean_values)), categories, rotation=45)
        plt.title('Avg. Across Folds')
        plt.tight_layout()
         # Save plot
        plt.savefig('Avg_Across_Folds_measures.png', dpi=300)
        plt.close()
    else:
        # Plot for each fold
        plt.figure(fold_index)
        fold_measures = measures[:3] + measures[4:]
        plt.bar(categories, fold_measures, color=colors)
        plt.title(f'Fold {fold_index}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        # Save plot
        plt.savefig(f'Fold_{fold_index}_measures.png', dpi=300)
        plt.close()

def plot_mae_pc(measures, mean_value = None, std_dev_value = None):
    """
    Plot and visualize 'MAE(PC)' across folds and for a specific fold.

    Args:
        measures (list): List of 'MAE(PC)' scores.
        mean_value (float): Mean value for MAE(PC).
        std_dev_value (float): Standard deviation value for 'MAE(PC)'.

    Returns:
        None: The function generates and saves the plot based on the specified parameters.
    """
    categories = ['Fold1', 'Fold2', 'Fold3', 'AcrossFolds']
    bar_positions = np.arange(4)
    plt.figure(5)
    plt.bar(bar_positions[:3], measures , color='orange')
    plt.bar(bar_positions[3], mean_value, yerr=std_dev_value, color='orange', capsize=5)
    plt.title('MAE(PC) Scores')
    plt.xticks(bar_positions, categories, rotation=45)
    plt.tight_layout()
    plt.savefig(f'MAE(PC)_Scores.png', dpi=300)
    plt.close()

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import mean_absolute_error
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import community.community_louvain as community_louvain
import os
def calculate_centralities(adj_matrix):
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError(f"Adjacency matrix is not square: shape={adj_matrix.shape}")
    print(f"Processing adjacency matrix of shape: {adj_matrix.shape}")

    G = nx.from_numpy_array(adj_matrix)
    partition = community_louvain.best_partition(G)

    # Calculate the participation coefficient with the partition
    pc_dict = participation_coefficient(G, partition)

    # Calculate averages of centrality measures
    pr = nx.pagerank(G, alpha=0.9)
    ec = nx.eigenvector_centrality_numpy(G, max_iter=100)
    bc = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    ns = np.array(list(nx.degree_centrality(G).values())) * (len(G.nodes()) - 1)
    acc = nx.average_clustering(G, weight=None)

    # Average participation coefficient
    pc_avg = np.mean(list(pc_dict.values()))

    return {
        'pr': np.mean(list(pr.values())),
        'ec': np.mean(list(ec.values())),
        'bc': np.mean(list(bc.values())),
        'ns': ns,
        'pc': pc_avg,
        'acc': acc
    }

def participation_coefficient(G, partition):
    # Initialize dictionary for participation coefficients
    pc_dict = {}

    # Calculate participation coefficient for each node
    for node in G.nodes():
        node_degree = G.degree(node)
        if node_degree == 0:
            pc_dict[node] = 0.0
        else:
            # Count within-module connections
            within_module_degree = sum(1 for neighbor in G[node] if partition[neighbor] == partition[node])
            # Calculate participation coefficient
            pc_dict[node] = 1 - (within_module_degree / node_degree) ** 2

    return pc_dict

def evaluate_all(true_hr_matrices, predicted_hr_matrices, output_path=path_eval_matrics):
    print(true_hr_matrices.shape)
    print(predicted_hr_matrices.shape)

    num_subjects = true_hr_matrices.shape[0]
    results = []

    for i in range(num_subjects):
        true_matrix = true_hr_matrices[i, :, :]
        pred_matrix = predicted_hr_matrices[i, :, :]

        print(f"Evaluating subject {i+1} with matrix shapes: true={true_matrix.shape}, pred={pred_matrix.shape}")

        if true_matrix.shape != pred_matrix.shape or true_matrix.shape[0] != true_matrix.shape[1]:
            print(f"Error: Matrix shape mismatch or not square for subject {i+1}: true={true_matrix.shape}, pred={pred_matrix.shape}")
            continue

        metrics = {
            'ID': i + 1,
            'MAE': mean_absolute_error(true_matrix.flatten(), pred_matrix.flatten()),
            'PCC': pearsonr(true_matrix.flatten(), pred_matrix.flatten())[0],
            'JSD': jensenshannon(true_matrix.flatten(), pred_matrix.flatten()),
        }

        true_metrics = calculate_centralities(true_matrix)
        pred_metrics = calculate_centralities(pred_matrix)

        for key in ['NS', 'PR', 'EC', 'BC', 'PC', 'ACC']:
            metrics[f'MAE in {key}'] = mean_absolute_error([true_metrics[key.lower()]], [pred_metrics[key.lower()]])

        results.append(metrics)

    df = pd.DataFrame(results)
    if not df.empty:
        # Check if the file exists to decide whether to write headers
        file_exists = os.path.isfile(output_path)

        df.to_csv(output_path, mode='a', header=not file_exists, index=False)
        print(f"Results appended to {output_path}.")
    else:
        print("No data to save.")



In [ ]:
from sklearn.model_selection import KFold
import time
import random
from memory_profiler import profile
import pandas as pd
import scipy.io
from scipy.io import loadmat

# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

"""
Main utility files
"""
class Argument():
    """
    Argument class to parse hyperparameters
    Please change the args during hyperparameter tuning
    """
    def __init__(self):
        super(Argument, self).__init__()
        self.epochs = 108
        self.lr_d = 2e-4
        self.lr_g = 4e-4
        self.splits = 3
        self.lmbda = 16
        self.lr_dim = 160
        self.hr_dim = 320
        self.hidden_dim = 100
        self.padding = 26
        self.mean_dense = 0.0
        self.std_dense = 0.01
        self.mean_gaussian = 0.0
        self.std_gaussian = 0.1
        self.dropout = 0
        self.ks = [0.9, 0.7, 0.6, 0.5]
        self.data_path = '' # path of input csv files

def convert_csv_to_mat(data_path = './'):
    """
    Convert CSV files to MATLAB (.mat) format and save.

    Args:
        data_path (str): The path to the directory containing CSV files. (Default is current directory)
    """

    if option == 0:
        lr_train = pd.read_csv(path_lr_data_tr)
        hr_train = pd.read_csv(path_hr_data_tr)

        lr_valid = pd.read_csv(path_lr_data_valid) # ignore thios - this doesnt matter - it is for consistency
        hr_valid = pd.read_csv(path_hr_data_valid) # ignore this - this doesnt matter  - it is for consistency

        lr_test = pd.read_csv(path_lr_data_valid)
        hr_test = pd.read_csv(path_hr_data_valid)


    if option == 1:
        lr_train = pd.read_csv(path_lr_data)
        hr_train = pd.read_csv(path_hr_data)

        lr_valid = pd.read_csv(path_lr_data_valid) # IGNORE this doesnt matter  - it is for consistency
        hr_valid = pd.read_csv(path_hr_data_valid) # IGNORE this doesnt matter  - it is for consistency

        lr_test = pd.read_csv(path_lr_data_test)
        hr_test = pd.read_csv(path_hr_data_test)

    scipy.io.savemat(data_path + 'lr_train.mat', {'data': lr_train})
    scipy.io.savemat(data_path + 'hr_train.mat', {'data': hr_train})
    scipy.io.savemat(data_path + 'lr_valid.mat', {'data': lr_valid})
    scipy.io.savemat(data_path + 'hr_valid.mat', {'data': hr_valid})

    scipy.io.savemat(data_path + 'lr_test.mat', {'data': lr_test})
    scipy.io.savemat(data_path + 'hr_test.mat', {'data': hr_test})


def load_data(data_path = './'):
    """
    Load data from CSV files converted to MATLAB (.mat) format.

    Args:
        data_path (str): The path to the directory containing the .mat files. (Default is current directory)

    Returns:
        lr_train_vector (pd.DataFrame): Vector containing data from 'lr_train.mat'.
        hr_train_vector (pd.DataFrame): Vector containing data from 'hr_train.mat'.
        lr_test_vector (pd.DataFrame): Vector containing data from 'lr_test.mat'.
    """
    convert_csv_to_mat(data_path) # load input csv files and store in .mat format

    # Load .mat into pd
    lr_train_data = loadmat(data_path + 'lr_train.mat')
    hr_train_data = loadmat(data_path + 'hr_train.mat')

    lr_valid_data = loadmat(data_path + 'lr_valid.mat')
    hr_valid_data = loadmat(data_path + 'hr_valid.mat')

    lr_test_data = loadmat(data_path + 'lr_test.mat')
    hr_test_data = loadmat(data_path + 'hr_test.mat')


    lr_train_vector = lr_train_data['data']
    hr_train_vector = hr_train_data['data']
    lr_valid_vector = lr_valid_data['data']
    hr_valid_vector = hr_valid_data['data']

    lr_test_vector = lr_test_data['data']
    hr_test_vector = hr_test_data['data']

    return lr_train_vector, hr_train_vector, lr_valid_vector, hr_valid_vector, lr_test_vector, hr_test_vector

def construct_dataset(lr_train_vector, hr_train_vector, lr_valid_vector, hr_valid_vector, lr_test_vector, hr_test_vector):
    """
    Construct numpy dataset from vectorized adjacency matrices.

    Args:
        lr_train_vector (pd.DataFrame): LR adjacency matrices for training.
        hr_train_vector (pd.DataFrame): HR adjacency matrices for training.
        lr_test_vector (pd.DataFrame): LR adjacency matrices for testing.

    Returns:
        lr_train_matrix_all_np (numpy.ndarray): Numpy array of size (n, 160, 160) containing anti-vectorized LR adjacency matrices for training.
        hr_train_matrix_all_np (numpy.ndarray): Numpy array of size (n, 268, 268) containing anti-vectorized HR adjacency matrices for training.
        lr_test_matrix_all_np (numpy.ndarray): Numpy array of size (n, 160, 160) containing anti-vectorized LR adjacency matrices for testing.
    """
    # Lists to store all anti-vectorized graphs
    lr_train_matrix_all = []
    hr_train_matrix_all = []
    lr_valid_matrix_all = []
    hr_valid_matrix_all = []

    lr_test_matrix_all = []
    hr_test_matrix_all = []


    # Anti-vectorize the adj matrix and append to a list
    for i in range(lr_train_vector.shape[0]):
        lr_train_matrix_all.append(MatrixVectorizer.anti_vectorize(lr_train_vector[i,:], 160, include_diagonal=False))
        hr_train_matrix_all.append(MatrixVectorizer.anti_vectorize(hr_train_vector[i,:], 268, include_diagonal=False))

    for i in range(lr_valid_vector.shape[0]):
        lr_valid_matrix_all.append(MatrixVectorizer.anti_vectorize(lr_valid_vector[i,:], 160, include_diagonal=False))
        hr_valid_matrix_all.append(MatrixVectorizer.anti_vectorize(hr_valid_vector[i,:], 268, include_diagonal=False))

    for i in range(lr_test_vector.shape[0]):
        lr_test_matrix_all.append(MatrixVectorizer.anti_vectorize(lr_test_vector[i,:], 160, include_diagonal=False))
        hr_test_matrix_all.append(MatrixVectorizer.anti_vectorize(hr_test_vector[i,:], 268, include_diagonal=False))

    # Convert list to Numpy
    lr_train_matrix_all_np = np.array(lr_train_matrix_all)
    hr_train_matrix_all_np = np.array(hr_train_matrix_all)

    lr_valid_matrix_all_np = np.array(lr_valid_matrix_all)
    hr_valid_matrix_all_np = np.array(hr_valid_matrix_all)

    lr_test_matrix_all_np = np.array(lr_test_matrix_all)
    hr_test_matrix_all_np = np.array(hr_test_matrix_all)

    return lr_train_matrix_all_np, hr_train_matrix_all_np, lr_valid_matrix_all_np, hr_valid_matrix_all_np, lr_test_matrix_all_np, hr_test_matrix_all_np

def cross_validation_3folds(args, train_adj, train_labels, valid_adj, valid_labels, cv):
    """
    Perform 3-fold cross-validation on the given data.

    Args:
        args (object): Input arguments.
        train_adj (numpy.ndarray): Array of adjacency matrices for training.
        train_labels (numpy.ndarray): Array of labels for training.
        cv (object): Cross-validation generator.

    Returns:
        measures_folds (list): Evaluation measures for each fold.
    """
    fold_index = 0
    measures_folds = []

    model = IWASAGSRNet(args)
    subjects_adj, val_adj, subjects_ground_truth, val_ground_truth = train_adj, valid_adj, train_labels, valid_labels
    train(model, subjects_adj, subjects_ground_truth, args)

    pred_matrices, gt_matrices = test(model, val_adj, val_ground_truth, args)


    # Save prediction results for every fold
    fold_outputs = pred_matrices.reshape(-1)
    print("========== Predictions ARE MADE! ==========")

    return fold_outputs

def predict_test(test_adj, args, model):
    """
    Predict the output for test adjacency matrices using the given model.

    Args:
        test_adj (numpy.ndarray): Array of test adjacency matrices.
        args (object): Input arguments.
        model (object): Model for prediction.

    Returns:
        total_outputs (numpy.ndarray): Predicted outputs for test adjacency matrices.
    """
    # Set the model to evaluation mode
    model.eval()
    output_list = []

    # Forward pass for prediction
    with torch.no_grad():
        for lr in test_adj:
            lr = torch.from_numpy(lr).type(torch.FloatTensor)
            outputs, _, _ = model(lr, args.lr_dim)
            outputs = unpad(outputs, args.padding)
            output_list.append(outputs)
        preds_list = np.asarray(output_list)
    return preds_list


def save_predictions(total_outputs, fold_num):
    """
    Save predictions into a CSV file with the required format.

    Args:
        total_outputs (numpy.ndarray): Predicted outputs.
        fold_num (int): Fold number for naming the CSV file.
    """

    # Create a DataFrame with ID and Predicted columns
    df = pd.DataFrame({'ID': range(1, len(total_outputs)+1), 'Predicted': total_outputs})
    # Save DataFrame to a CSV file
    df.to_csv(f'predictions_fold_{fold_num}.csv', index=False)

def main():
    args = Argument()

    print("Please Specify the Data Path of the Input CSV Files! (currently using the default path under ./data)")
    print("======== Loading Data... ========")
    lr_train_vector, hr_train_vector, lr_valid_vector, hr_valid_vector, lr_test_vector, hr_test_vector = load_data(args.data_path)
    train_adj, train_labels, valid_adj, valid_labels, test_adj, test_labels = construct_dataset(lr_train_vector, hr_train_vector, lr_valid_vector, hr_valid_vector, lr_test_vector, hr_test_vector)
    print("======== Data Loaded! ========")

    print("========== Start Fold Cross Validation ==========")
    #cv = KFold(n_splits=args.splits, shuffle=True, random_state = random_seed)
    cv = None
    start_time = time.time() # timer for cv run time
    #measures_folds = cross_validation_3folds(args, train_adj, train_labels, valid_adj, valid_labels, cv)
    #end_time = time.time()
    #elapsed_time = end_time - start_time
    #print(f"Total training time for 3F-CV: {elapsed_time} seconds")

    # Plot measures across all folds
    #mean_values = np.mean(measures_folds, axis=0)
    #std_dev_values = np.std(measures_folds, axis=0)
    #plot_measures(avg = True, mean_values = mean_values, std_dev_values = std_dev_values)

    # Plot MAE(pc) measure
    #mae_pc_folds = [measures[3] for measures in measures_folds]
    #plot_mae_pc(mae_pc_folds, mean_value = mean_values[3], std_dev_value = std_dev_values[3])

    print("========== Generate Final Model with All Training Samples ==========")
    final_model = IWASAGSRNet(args)
    train(final_model, train_adj, train_labels, args)
    final_output = predict_test(test_adj, args, final_model)
    #save_predictions(final_output, 'all')

    metrics = evaluate_all(
    test_labels, final_output
    )

if __name__ == "__main__":
    main()